In [ ]:
%tensorflow_version 2.8

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!python3 --version

Python 3.10.12


In [ ]:
import os

In [ ]:
import tensorflow as tf

In [ ]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
!nvidia-smi

Thu Mar  7 01:08:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0              27W /  70W |    105MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!python -m pip install mediapipe
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker



Python 3.10.12


In [ ]:

from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

In [ ]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    # Visible devices must be set at program startup
    print(e)

In [ ]:

gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:

import os

# get the current working directory
current_working_directory = os.getcwd()

# print output to the console
print(current_working_directory)

/content


In [ ]:

current_working_directory

'/content'

In [ ]:
datasets_folder = os.path.join(current_working_directory, "drive/MyDrive/R2045 FTC Team #19829/F. Team Prop/Dataset-BlackEdges/")

In [ ]:
train_dataset_path = os.path.join(datasets_folder, "train/")
validation_dataset_path = os.path.join(datasets_folder, "valid/")
test_dataset_path = os.path.join(datasets_folder, "test/")


In [ ]:
len(train_dataset_path)

83

In [ ]:

train_dataset_path

'/content/drive/MyDrive/R2045 FTC Team #19829/F. Team Prop/Dataset-BlackEdges/train/'

In [ ]:
train_data = object_detector.Dataset.from_pascal_voc_folder(train_dataset_path, cache_dir='/tmp/od_data/train')
validation_data = object_detector.Dataset.from_pascal_voc_folder(validation_dataset_path, cache_dir='/tmp/od_data/validation')


In [ ]:

len(train_data)

760

In [ ]:

len(validation_data)

211

In [ ]:
test_data = object_detector.Dataset.from_pascal_voc_folder(test_dataset_path, cache_dir='/tmp/od_data/test')

In [ ]:
len(test_data)
# len(train_data)

123

In [ ]:
spec = object_detector.SupportedModels.MOBILENET_V2
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)
model_options=object_detector.ModelOptions(
    l2_weight_decay = 3e-05
)

In [ ]:
spec


<SupportedModels.MOBILENET_V2: functools.partial(<class 'mediapipe_model_maker.python.vision.object_detector.model_spec.ModelSpec'>, downloaded_files=DownloadedFiles(path='object_detector/mobilenetv2_i256', url='https://storage.googleapis.com/tf_model_garden/vision/qat/mobilenetv2_ssd_coco/mobilenetv2_ssd_i256_ckpt.tar.gz', is_folder=True), checkpoint_name='ckpt-277200', input_image_shape=[256, 256, 3], model_id='MobileNetV2', min_level=3, max_level=7)>

In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

TypeError: ObjectDetector.__init__() missing 3 required positional arguments: 'model_spec', 'hparams', and 'model_options'

In [ ]:
model.export_labels(export_dir=datasets_folder)

In [ ]:
loss, coco_metrics = model.evaluate(test_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")


31/31 [==============================] - 3s 48ms/step - total_loss: 0.1798 - cls_loss: 0.1068 - box_loss: 3.7265e-04 - model_loss: 0.1255


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.07s).
Accumulating evaluation results...
DONE (t=0.14s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.887
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.887
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.914
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.914
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.914
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000


In [ ]:
model.export_tflite(export_dir=datasets_folder, tflite_filename = 'model.tflite',)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
/usr/local/lib/python3.10/dist-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tensorflow.python.ops.init_ops_v2.Constant'>, which may lead to improper serialization.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for obj